<a href="https://colab.research.google.com/github/yungGenos/cronometro/blob/main/atividade_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#-----------------------
# RA 421110705 - kaique vinicius souza genonadio da silva
# RA 

In [ ]:
# Pacotes necessários
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

# import xgboost

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor 
from sklearn.ensemble import AdaBoostRegressor
from lightgbm import LGBMRegressor

# Medidas de avaliação
from sklearn import metrics

In [ ]:
df_taxi_ny_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   id                  1458644 non-null  object 
 1   vendor_id           1458644 non-null  int64  
 2   pickup_datetime     1458644 non-null  object 
 3   dropoff_datetime    1458644 non-null  object 
 4   passenger_count     1458644 non-null  int64  
 5   pickup_longitude    1458644 non-null  float64
 6   pickup_latitude     1458644 non-null  float64
 7   dropoff_longitude   1458644 non-null  float64
 8   dropoff_latitude    1458644 non-null  float64
 9   store_and_fwd_flag  1458644 non-null  object 
 10  trip_duration       1458644 non-null  int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 122.4+ MB


In [ ]:
# Listando as colunas do arquivo de dados
df_taxi_ny_train.columns

Index(['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag',
       'trip_duration'],
      dtype='object')

In [ ]:
# Conexão com o drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Carrega a biblioteca pandas e atribui um alias chamado "pd"
import pandas as pd

In [ ]:
# Deixar colunas e linhas aparentes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Faz a leitura do arquivo train.csv 
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/taxi_ny_train.csv')

In [ ]:
#1Pesquise o significado dos hiperparâmetros tanto para o Random Forest quanto para o XGBOOST

#Os hiperparâmetros (hyperparameters) são ajustes que podem ser definidos antes do treinamento do modelo e que têm um impacto significativo no desempenho do modelo. O Random Forest e o XGBoost são algoritmos de aprendizado de máquina que possuem hiperparâmetros diferentes. A seguir, vou explicar os hiperparâmetros mais comuns para cada um deles:

#Hiperparâmetros do Random Forest:
#n_estimators: O número de árvores de decisão no modelo. Valores maiores geralmente melhoram o desempenho, mas aumentam o tempo de treinamento e o uso de memória.
#min_samples_split: O número mínimo de amostras necessárias para dividir um nó interno. Valores menores podem levar a uma árvore mais complexa e overfitting, enquanto valores maiores podem levar a uma árvore menos flexível.
#min_samples_leaf: O número mínimo de amostras necessárias em uma folha. Valores menores podem levar a overfitting, enquanto valores maiores podem levar a uma árvore menos flexível.
#max_features: O número máximo de recursos que cada árvore é permitida a considerar para fazer uma divisão. Valores menores podem reduzir a variância, mas podem aumentar o viés.

#Hiperparâmetros do XGBoost:
#learning_rate: O tamanho do passo de atualização em cada iteração enquanto se movimenta em direção ao mínimo da função de perda. Valores menores reduzem o viés, mas aumentam o tempo de treinamento.
#max_depth: A profundidade máxima de cada árvore de decisão. Valores maiores geralmente aumentam o desempenho, mas também podem aumentar o risco de overfitting.
#min_child_weight: O número mínimo de amostras necessárias em cada nó. Valores maiores podem levar a uma árvore mais conservadora, enquanto valores menores podem levar a overfitting.
#subsample: A fração de amostras a serem usadas para cada árvore de decisão. Valores menores podem reduzir a variância, mas podem aumentar o viés.
#colsample_bytree: A fração de recursos a serem usados para cada árvore de decisão. Valores menores podem reduzir a variância, mas podem aumentar o viés.
#Estes são apenas alguns dos hiperparâmetros disponíveis para cada modelo. É importante destacar que a escolha de valores apropriados para esses hiperparâmetros pode ser um processo complexo e envolve experimentação e ajuste fino.

In [ ]:
#2 Justifique a escolha dos hiperparâmetros com a explicação 

#Ao usar o hiperparâmetro random forest, é possível escolher um número aleatório de árvores dentro de um determinado intervalo. Esse método pode ajudar a encontrar um número ótimo de árvores que equilibre a precisão do modelo com sua eficiência. 
#No entanto, temos  em mente que a escolha do número ideal de árvores pode variar dependendo do conjunto de dados e do problema específico que estamos tentando resolver. Portanto, 
#é sempre importante experfrom sklearn.ensemble import RandomForestClassifie



In [ ]:
#3 Altere o percentual de treino e teste

# Separando as informações em y e x
y = df_taxi_ny_train['trip_duration']
X = df_taxi_ny_train.drop('trip_duration', axis=1)

# Separando as informações do arquivo de dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Criando o objeto da regressão Random Forest com alguns hiperparâmetros 
rf_regressor = RandomForestRegressor(n_estimators=50, max_depth=10, min_samples_split=5, min_samples_leaf=2, max_features='sqrt', random_state=42)

# modelo
rf_regressor.fit(X_train, y_train)

# teste
y_pred = rf_regressor.predict(X_test)


In [ ]:
#4 Aplique os algoritmos com 3 hiperparâmetros
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV

# Leitura do arquivo CSV com pandas
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/taxi_ny_train.csv')

# Separando as features das labels
X = df.drop('trip_duration', axis=1)
y = df['trip_duration']

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Definindo a grade de hiperparâmetros para busca aleatória
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth}

# Criando o objeto da regressão Random Forest
rf_regressor = RandomForestRegressor(random_state=42)

# Instanciando o objeto de busca aleatória
rf_random = RandomizedSearchCV(estimator = rf_regressor, 
                               param_distributions = random_grid, 
                               n_iter = 100, 
                               cv = 3, 
                               verbose=2, 
                               random_state=42, 
                               n_jobs = -1)

# Treinando o modelo com busca aleatória de hiperparâmetros
rf_random.fit(X_train, y_train)

# Exibindo os melhores parâmetros encontrados
print(rf_random.best_params_)

# Treinando o modelo com os melhores hiperparâmetros
best_rf = RandomForestRegressor(n_estimators=500,
                                 max_features='sqrt',
                                 max_depth=50,
                                 random_state=42)

best_rf.fit(X_train, y_train)

# Avaliando o modelo nos dados de teste
y_pred = best_rf.predict(X_test)


Fitting 3 folds for each of 100 candidates, totalling 300 fits


ValueError: ignored